In [1]:
# !git clone https://github.com/vietawake/ERFModel
# !mv -v ERFModel/* .

In [2]:
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
from torchvision import transforms
from models.erfnet_road import ERFNet
import cv2
from inference import preprocess, segmentation, visualize

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
np.random.seed(50)
torch.manual_seed(50)

if torch.cuda.is_available():
    torch.cuda.manual_seed(50)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
net = ERFNet(num_classes=4)

In [5]:
checkpoint = torch.load('./pretrained_models/ERF_epoch_39_acc_0.7864.pt',map_location=device)
multigpus = True
for key in checkpoint:  # check if the model was trained in multiple gpus
    if 'module' in key:
        multigpus = multigpus and True
    else:
        multigpus = False
if multigpus:
    net = torch.nn.DataParallel(net)
net.load_state_dict(checkpoint)
net.to(device)
net.eval()

ERFNet(
  (encoder): Encoder(
    (initial_block): DownsamplerBlock(
      (conv): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layers): ModuleList(
      (0): DownsamplerBlock(
        (conv): Conv2d(16, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): non_bottleneck_1d(
        (conv3x1_1): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (conv1x3_1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (conv3x1_2): Conv2d(64, 64, 

In [6]:
input_path = 'freespace_lanemarking_segmentation_data_test/img_test/autopilot_test_0001_20101230_083418_000160.png'
img = Image.open(input_path)
origin_height = img.size[1]
origin_width = img.size[0]
preprocess_img = preprocess(img, height=360, width=640)

seg_map = segmentation(preprocess_img, origin_height, origin_width, net, device)
print(seg_map.shape)
overlaid_img = visualize(seg_map, np.asarray(img))

combined_img = np.concatenate((np.asarray(img), overlaid_img), axis=1)

cv2.imwrite('ouput.jpg', combined_img)


(1080, 1920)


True